# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question, 
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown

In [23]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_BASE_URL = "http://localhost:11434/v1"

PROMPT_FILE = "system_prompt.txt"
try:
    with open(PROMPT_FILE, "r", encoding="utf-8") as f:
        SYSTEM_PROMPT = f.read().strip()
except FileNotFoundError:
    SYSTEM_PROMPT = "You are a helpful assistant that can answer questions about technical topics."


In [24]:
# set up environment
load_dotenv(override=True)
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:3]}")
else:
    print("OpenRouter API Key not set (and this is optional)")

OpenRouter API Key exists and begins sk-


In [25]:
# here is the question; type over this to ask something new

# question = """ Explain the theory of relativity.  """
# question = """  Does gravity affect more than just objects?   """
# question = """ What is the difference between a black hole and a white hole? """
# question = "Wetin be hyper-inflation?  "
question = " How person go take know when to eat if e no dey feel hunger?"


In [ ]:
# Get gpt-4o-mini to answer, with streaming

openrouter_url = "https://openrouter.ai/api/v1"
openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)
stream = openrouter.chat.completions.create(
        model=MODEL_GPT, messages=[
        {"role": "system", "content": SYSTEM_PROMPT}, 
        {"role": "user", "content": question}], 
        reasoning_effort="minimal", stream=True)

parts = []
out = display(Markdown(""), display_id=True)
for chunk in stream:
    delta = chunk.choices[0].delta
    if hasattr(delta, "content") and delta.content is not None:
        parts.append(delta.content)
        out.update(Markdown("".join(parts)))

In [ ]:
# Get Llama 3.2 to answer
import urllib.request

def ollama_is_running(base_url="http://localhost:11434"):
    try:
        with urllib.request.urlopen(f"{base_url}/api/tags", timeout=2) as r:
            return r.status == 200
    except OSError:
        return False

if ollama_is_running():
    print("Ollama is running")
    ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
    response = ollama.chat.completions.create(
    model="llama3.2", 
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question}
    ])
    display(Markdown(response.choices[0].message.content))
else:
    print("Ollama is not running — start it with: ollama serve")

